In [1]:
from utils import globo , xdv 
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time , os , cv2

os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ["CUDA_VISIBLE_DEVICES"]="3"

/raid/DATASETS/.zuble/vigia /raid/DATASETS/anomaly


#### VSWIN TRANSFORMER

In [2]:
model = tf.saved_model.load("/raid/DATASETS/.zuble/vigia/zurgb11/.pretrained/swin_tiny_patch244_window877_kinetics400_1k_1")

In [3]:
shape_of_input = [1,3,32,224,224]   # [batch_size, channels, frames, height, width]
dummy_video = tf.random.normal(shape_of_input)

output = model(dummy_video)

print(output.shape)


(1, 768, 16, 7, 7)


#### XDV INFO

In [ ]:
from utils import xdv , yammet , globo
import numpy as np , os

yammet = yammet.Yammet(globo.CFG_YAMMET)


fn, labels, tot_frames = xdv.load_train_valdt_npy('train',4000)

In [ ]:
i = 0
for f in fn:
    
    l = f.split("label_")[1].replace(".mp4","")
    
    if l != 'A':
        print("\n$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n\t\t",\
        os.path.basename(f))
        p_yam_arr_max = yammet.get_sigmoid(f,ef=4000,debug=True)
        for kkk,anom_yam_k in enumerate(globo.CFG_YAMMET["anom_labels_i"]):
            if p_yam_arr_max[anom_yam_k] >= 0.4:
                print('\t\t-TP- {}: {:.4f}'.format(np.array(globo.CFG_YAMMET['labels'])[anom_yam_k], p_yam_arr_max[anom_yam_k]))
                print("\t\tCHECK")
                break
    '''else:
        fp_flag = False
        p_yam_arr_max = yammet.get_sigmoid(f,debug=True)
        for kkk,anom_yam_k in enumerate(globo.CFG_YAMMET["anom_labels_i"]):
            if p_yam_arr_max[anom_yam_k] >= 0.4:
                print('\n\t\t-FP- {}: {:.4f}'.format(np.array(globo.CFG_YAMMET['labels'])[anom_yam_k], p_yam_arr_max[anom_yam_k]))
                fp_flag = True
                break          
        if not fp_flag: print("\n\t\tCHECK") '''       

In [ ]:
xdv.get_info_from_infotxt()

In [ ]:
import numpy as np
frame = []
for i in range(10):
    frame.append(np.ones((155,155,3),np.float32))
t1 = np.stack(frame)
t2 = np.stack(frame,axis=0)

### mobile net inference

In [ ]:
model = tf.saved_model.load("/raid/DATASETS/.zuble/vigia/zurgb11/model/model/1683929580.7657886_mobilenetv2_fp16_relu_0.0002_sgd_0_4_4000")

In [ ]:
@tf.function
def vas_predict(model,x):
    return model(x, training=False)    

def get_n_frames(video_path, n):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = [i * total_frames // n for i in range(n)]

    for i in range(total_frames):
        ret, frame = cap.read()
        if i in frame_indices:
            frame = cv2.resize(frame, (224, 224) )
            frame = np.array(frame)/255.0
            frames.append(frame)
    cap.release()
    
    return np.expand_dims(np.array(frames).astype(np.float32), 0)

def test_model():
    test_fn, *_ = xdv.test_files()
    filtered_test_fn = [fn for fn in test_fn if 'label_A' not in fn]
    for i,fn in enumerate(test_fn):
            print(fn)
            if i == 5:break
            inp = get_n_frames(fn, 500)
            t = time.time()
            output = vas_predict(model,inp)[0][0].numpy()
            print(f"Output shape: {output.shape}")
            tt = time.time()
            print(output,str(tt-t))

test_model()
